# Loss Function

もうちょっといいタイトル無かったのか……．

## 二乗和誤差（mean squared error）
数式にするとこんな感じ．

$$y_k = 1/2*\sum_{k} (y_k-t_k)^2$$

In [1]:
import numpy as np

In [2]:
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t) ** 2)

In [3]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0,0,0,0,0,0,0]

In [4]:
mean_squared_error(np.array(y), np.array(t))

0.097500000000000031

In [5]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.5, 0.0] # 7番目が最も高い
t = [0, 0, 1, 0,0,0,0,0,0,0] # 2番目が最も高い

In [6]:
mean_squared_error(np.array(y), np.array(t))

0.72250000000000003

損失関数の値が小さい　→　教師データとの誤差が小さくなる．


上の例の方が出力される値が小さい　→　誤差が小さい

## cross entropy error

$$E = -\sum_{k} t_k log(y_k)$$

Tkが正解ラベル，Ykが出力．このときTkは正解が1，その他は0（One-hot 表現）．

→　正解のラベルに対応する部分の自然対数を出力するだけ（ほかは全部0なので）．

訓練データ1件ごとに損失関数を計算．その総和が小さくなる　→　いろんなデータに対して正しく分類できるようになることを示す．

なので，損失関数の総和が小さくなるようにパラメータを調整していけばよい．交差エントロピー誤差について式にするとこんな感じ．

$$E = -1/N \sum_{n} \sum_{k} t_nk log(y_nk)$$

データの数がN個，1/Nで正規化している．

とはいえ全部の損失関数を計算するのだるい　→　分かる．というか，過学習に繋がりかねない　→　分かる．

全体から無作為に選んだ特定の数のデータに対して損失を計算して，それを指標として学習する．

これを *ミニバッチ学習* と言う．

In [1]:
import sys, os
import numpy as np
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

In [2]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
# ネット環境がクソなせいでDLできないぃいいいいいい

In [3]:
print(x_train.shape)

(60000, 784)


In [5]:
print(t_train.shape)

(60000, 10)


In [6]:
# Numpyの関数で訓練データからrandomに10枚だけ抜き出す
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) # 60000から10個ランダムに選ぶ
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [7]:
def cross_entropy_error(y, t):
    """
    y = ニューラルネットワークの出力
    t = 教師データ
    1次元データのときはコケるからreshapeしてなんとかしているらしい．
    """
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y)) / batch_size

↑ return文の中身はトレーニングデータの内容によって変化する．
one-hot表現の場合：普通に自然対数を取る（上の要な場合）．
ラベル表現の場合：ちょっと変わってくる．↓のようになる．

In [7]:
# -np.sum(t * np.log(y[np.arange(batch_size), t])) / batch_size

tは正解のラベルデータが配列で入っている．
np.arange(batch_size)で，0〜batch_size-1の配列を作るもの．
y[np.arange(batchi_size), t]　→　各正解データのラベルに対応するニューラルネットワークの出力を抽出．